In [4]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import nltk
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_validate
import numpy as np
import re
from nltk.corpus import stopwords
stop_words=set(stopwords.words('english'))
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()
import string

In [5]:
#loading the data

import pandas as pd
df=pd.read_csv("/Users/divyaj_podar/Desktop/final_set_label.csv",header=0)
df.head()

,Channel Name,Brand,Model,URL,Published_date,Views,Likes,Dislikes,user,commentText,likes,label
0,Unbox_Therapy,samsung,s10,https://www.youtube.com/watch?v=7YKjLzlc-hY&t=3s,21-Feb-19,"4,421,639",104000.0,3800.0,Unbox Therapy,Your favorite smartphone right now... Galaxy S...,5783.0,1.0
1,Unbox_Therapy,samsung,s10,https://www.youtube.com/watch?v=7YKjLzlc-hY&t=3s,21-Feb-19,"4,421,639",104000.0,3800.0,Neil.is.FAMOUS,I caved today and got an s10 plus for my tiny ...,0.0,1.0
2,Unbox_Therapy,samsung,s10,https://www.youtube.com/watch?v=7YKjLzlc-hY&t=3s,21-Feb-19,"4,421,639",104000.0,3800.0,Carter Mickens,Samsung: Give them more to do and see\nApple: ...,0.0,1.0
3,Unbox_Therapy,samsung,s10,https://www.youtube.com/watch?v=7YKjLzlc-hY&t=3s,21-Feb-19,"4,421,639",104000.0,3800.0,Crookie 2k,Yes it is the phone,0.0,1.0
4,Unbox_Therapy,samsung,s10,https://www.youtube.com/watch?v=7YKjLzlc-hY&t=3s,21-Feb-19,"4,421,639",104000.0,3800.0,Tim Hex,Have u guys bought this phone I just wanna kno...,0.0,1.0


In [6]:
# segregation of data by different cell phones
df_6t=df[df['Model']=='6t']
df_s10=df[df['Model']=='s10']
df_pixel3=df[df['Model']=='pixel3']
df_xs=df[df['Model']=='xs']

print("Shape of Oneplus 6t Dataframe: ",df_6t.shape)
print("Shape of Samsung s10 Dataframe: ",df_s10.shape)
print("Shape of iphone xs Dataframe: ",df_xs.shape)
print("Shape of Pixel 3 Dataframe: ",df_pixel3.shape)

df_xs.iloc[199]

Shape of Oneplus 6t Dataframe:  (25418, 12)
Shape of Samsung s10 Dataframe:  (28987, 12)
Shape of iphone xs Dataframe:  (24239, 12)
Shape of Pixel 3 Dataframe:  (30655, 12)


Channel Name                                          Unbox_Therapy
Brand                                                         apple
Model                                                            xs
URL                https://www.youtube.com/watch?v=XFckmtISfJk&t=2s
Published_date                                            22-Sep-18
Views                                                     4,838,265
Likes                                                        156000
Dislikes                                                       6100
user                                            Muhaimin Saddaramil
commentText       pls can I have one, just for my incoming birth...
likes                                                             0
label                                                             1
Name: 35692, dtype: object

In [7]:
##################################### Samsung s10 #######################################

In [8]:
training_set=df_s10.iloc[:1000]    # a set with labeled data on the basis of relevance for training
testing_set=df_s10[1000:1200]     # a set with labeled data on the basis of relevance for testing
text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                             ('clf', MultinomialNB())])

#finding the best parameters
parameters={'tfidf__min_df' : [1,2,3],
                'tfidf__stop_words' : [None,"english"],
                'clf__alpha' : [0.5,1,2]}        

metric =  "f1_macro"   

gs_clf = GridSearchCV(text_clf, param_grid=parameters,scoring=metric, cv=5)

gs_clf = gs_clf.fit(training_set["commentText"], training_set["label"])    

for param_name in gs_clf.best_params_:
    print(param_name,": ",gs_clf.best_params_[param_name])

####################### Testing results on the trained "s10" data #######################

#generating the tfidf matrix
tfidf_vect = TfidfVectorizer(stop_words='english', min_df=2)

dtm = tfidf_vect.fit_transform(training_set["commentText"])

# train a multinomial naive Bayes model using the testing data
clf = MultinomialNB(alpha=0.5).fit(dtm, training_set["label"])

dtm_testing=tfidf_vect.transform(testing_set["commentText"])

labels=sorted(training_set["label"].unique())

# predict the news group for the test dataset
predicted=clf.predict(dtm_testing)

#printing the classification report 
print(classification_report(testing_set["label"], predicted))

clf__alpha :  0.5
tfidf__min_df :  2
tfidf__stop_words :  english
              precision    recall  f1-score   support

         0.0       0.96      0.30      0.46        80
         1.0       0.68      0.99      0.81       120

   micro avg       0.71      0.71      0.71       200
   macro avg       0.82      0.65      0.63       200
weighted avg       0.79      0.71      0.67       200



In [9]:
##################################### Iphone XS #######################################

In [10]:
training_set_xs=df_xs.iloc[:800]
testing_set_xs=df_xs.iloc[800:999]
text_clf_xs = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', MultinomialNB())])

parameters_xs={'tfidf__min_df' : [1,2,3],
            'tfidf__stop_words' : [None,"english"],
            'clf__alpha' : [0.5,1,2]}        

metric_xs =  "f1_macro"   

gs_clf_xs = GridSearchCV(text_clf_xs, param_grid=parameters_xs,scoring=metric_xs, cv=5)
gs_clf_xs = gs_clf.fit(training_set_xs["commentText"], training_set_xs["label"])    
    
for param_name in gs_clf_xs.best_params_:
    print(param_name,": ", gs_clf_xs.best_params_[param_name])

tfidf_vect_xs = TfidfVectorizer(stop_words=None, min_df=3)
dtm_xs = tfidf_vect_xs.fit_transform(training_set_xs["commentText"])
clf_xs = MultinomialNB(alpha=0.5).fit(dtm_xs, training_set_xs["label"])

####################### Testing results on the trained "xs" data #######################
dtm_testing_xs=tfidf_vect_xs.transform(testing_set_xs["commentText"])
labels=sorted(training_set_xs["label"].unique())
predicted_xs=clf_xs.predict(dtm_testing_xs)

print(classification_report(testing_set_xs["label"], predicted_xs))

clf__alpha :  0.5
tfidf__min_df :  3
tfidf__stop_words :  None
              precision    recall  f1-score   support

         0.0       0.77      0.44      0.56        68
         1.0       0.76      0.93      0.84       131

   micro avg       0.76      0.76      0.76       199
   macro avg       0.77      0.69      0.70       199
weighted avg       0.76      0.76      0.74       199



/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
